# Sentiment Adjusted Stock Prediction
By:

Date: /2025

With the virtually infinite amount of information floating around on social media about current world affairs, it would be nice to be able to automatically parse the information and get the data you need related to which investments you should buy. We can solve this problem by building a program to automatically scrape the data and derive key details from it using AI.

We believe that we can get a better picture of what stocks will be if we can combine 2 things, well optimized stock data and a twitter api. The first part of this report will focus on the:

# Stock Data Collection

Large detailed stock datasets were hard to come by, so we had to come up with a strategy to build a large enough dataset within a few acceptance criteria:

- A large array of stocks, given in this project as "stock_tickers.csv" or S
- Stocks that were going to stay relatively stable, for this I selected a list of the companies with the largest market cap descending, and collected as large a data set as possible (max(size(S)))
- Stocks that were freely available with the data from Alpha Vantage (the API used in this project) set of Alpha Vantage given A, condition S in A
- Stocks will be picked from the top of the Nasdaq top stock list S_i = N_i, i = {1,2,3,\dots,n}
- Number of pulls the API will allow me to do from Wall Street close on Friday to open on Monday (don't know how to express this one as the API throttling seems to happen at random)

Provided mathematically as:

max(size(S))

s.t.

S in A

S_i = N_i, i = {1,2,3,\dots,n}



## Objective Function max(size(S))
This 